# Part a): Ordinary Least Square on the Franke function
In this notebook, we generate a dataset by sampling the Franke function on the rectangle $[0,1]\times[0,1]$, both with and without the presence of added noise. We then try fitting a polynomial function to these dataset, and evaluate how well we are able to approximate the data.

In [3]:
import numpy as np
import os
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

# Import local modules
from src.data.generate_data import FrankeFunction
from src.models.models import OLS
from src.features.polynomial import PolynomialFeatures
from src.evaluation.evaluation import mse, r_squared

We begin by creating the features. Here, we use the class $\texttt{PolynomialFeatures}$ in $\texttt{src.features.polynomial}$.

In [4]:
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)
x, y = np.meshgrid(x, y)
x, y = x.ravel(), y.ravel()
pf = PolynomialFeatures(5)
X = pf.fit_transform(np.c_[x.reshape((-1, 1)), y.reshape((-1, 1))])

Now, we create the target values by sampling the Franke Function. By varying the noise term, we obtain different data sets.

In [5]:
# Compute mse for varying noise:
ols = OLS()
noises = np.arange(0, 3.0, 0.2)
print('%-10s%-10s%-10s' %('Noise', 'MSE', 'R squared'))
for noise in noises:
    z = FrankeFunction(x, y, noise=noise, seed=43)
    ols.fit(X, z)
    predictions = ols.predict(X)
    mean_squared_error = mse(z, predictions)
    r_s = r_squared(z, predictions)
    print('%-10.2f%-10.3f%-10.8f' %(noise, mean_squared_error, r_s))

Noise     MSE       R squared 
0.00      0.002     0.99993004
0.20      0.038     0.99919371
0.40      0.152     0.99842182
0.60      0.343     0.99805820
0.80      0.610     0.99788163
1.00      0.955     0.99778555
1.20      1.377     0.99772790
1.40      1.877     0.99769059
1.60      2.453     0.99766500
1.80      3.107     0.99764662
2.00      3.838     0.99763293
2.20      4.646     0.99762243
2.40      5.531     0.99761417
2.60      6.493     0.99760753
2.80      7.533     0.99760211


We see a how an increase in noise increases the mean square error, but that the R squared statistic remains fairly close to 1.